In [58]:
import csv
from lxml import html
import pandas as pd
import numpy as np
import nltk
import string
import re
from collections import defaultdict
from nltk.stem.snowball import EnglishStemmer
from pickle import load
from sklearn.pipeline import Pipeline, make_pipeline

# Loading models

In [2]:
tfidf = load(open('tfidf', 'rb'))
model = load(open('model', 'rb'))

# Loading stopwords

In [3]:
nltk.download('stopwords')
tokenizer = nltk.RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mlmaster/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading Data

In [4]:
x = pd.read_csv('QueryResults.csv')[0:5]
title = x['Title']
body = x['Body']

In [22]:
to_keep = open("to_keep.csv", 'r')
to_keep = csv.reader(to_keep)
to_keep = list(to_keep)

sw = open("sw.csv", 'r')
sw = csv.reader(sw)
sw = list(sw)

# Building the pipeline

In [79]:
def pipe(title, body):
    body = title + body
    body = title + body
    print(body)
    parser = lambda x: html.fromstring(x).text_content()
    body = list(map(parser, body))
    print(body)
    body.replace(r'\n', '', regex=True, inplace=True)
    
    # There is still some html tags, let's remove them with a regex
    TAG_RE = re.compile(r'<[^>]+>')
    tags_re = lambda x: TAG_RE.sub('', x)
    body = list(map(tags_re, body))
    print(body)
    # Init stemmer and tokenizer
    stemmer = EnglishStemmer()
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    
    # Loop over data to remove stop words and keep only the good words
    desc = []
    i = 0
    for row in body:
        if i%2000 == 0:
            print("Performing Step ", i, "/", len(body))
        token = tokenizer.tokenize(row.lower())
        description = [stemmer.stem(w) for w in token if not w in sw and w in to_keep]
        desc.append(description)
        i += 1
    body_cleaned = [' '.join(map(str, l)) for l in desc]
    
    return body_cleaned

In [80]:
body

0    <p>The below code only seems to work on the fi...
1    <p>I have two questions:</p>\n\n<ol>\n<li><p>I...
2    <p>I am trying to connect Sql server and MySql...
3    <p>I use angular 2 and bootstrap 4, There is a...
4    <p>I have a xml generated from SharePoint and ...
Name: Body, dtype: object

In [81]:
pipe(title, body)

0    Macro only running through first resultMacro o...
1    Set limitation on gap tolerance and run timeSe...
2    Connect two different Database with one applic...
3    bootstrap 4 navbar disappears when resizing sc...
4    xml transform XSLT with namespacesxml transfor...
dtype: object
['Macro only running through first resultMacro only running through first resultThe below code only seems to work on the first result. I have to re-run it for the other results to be removed. Could anyone take a look and tell me why please? Thanks\n\nI\'ve tried copy and pasting the code several times to compensate but it gives me an error about duplicate code\n\nDim cell As Range\nFor Each cell In [AE1:AE2000]\n    If cell.Value = "REMOVE" Then Range(cell.Offset(0, -5).Address & ":" & cell.Offset(0, 0).Address).Delete Shift:=xlUp\nNext cell\n\n', 'Set limitation on gap tolerance and run timeSet limitation on gap tolerance and run timeI have two questions:\n\n\nIn IBM ILOG CPLEX, I wrote an OPL project

AttributeError: 'list' object has no attribute 'replace'

In [12]:
pipe = Pipeline([('tf-idf', tfidf), ('Model', model)])